In [13]:
import pandas as pd
import numpy as np

In [14]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [422]:
class Prepack:
    """Data preparation library v0.1""" 
    
    def __init__(self):
        pass
    
    @staticmethod
    def isnan(v):
        import math as m
        if isinstance(v, float) and m.isnan(v):
            return True
        else:
            return False
    
    @staticmethod    
    def list_concat(lst1, lst2):

        lst3 = []
        size1 = len(lst1)
        size2 = len(lst2)
        #здесь будет размер самого длинного списка
        size_max = max(size1,size2)

        #вытаскиваем и делаем строковыми значения из обоих списков
        for i in range(size_max):
            ss = ''
            if i < size1 and not Prepack.isnan(lst1[i]):
                s = str(lst1[i])
                if len(s) > 0:
                    ss += s
            if i < size2 and not Prepack.isnan(lst2[i]):
                s = str(lst2[i])
                if len(s) > 0:
                    if len(ss) > 0:
                        ss += ' '
                    ss += s
            #кладем все в список в виде объединенных значений
            lst3 += [ss]
        return lst3
    
    @staticmethod 
    def p2c(lst):
        import numpy as np
        length = len(lst)
        max_len = max(len(repr(el)) for el in lst)
        # делаем кортеж со значениями из списка
        cols = tuple(lst)
        # делаем массив из кортежа названий и диапазона от 0 до length, .T транспонирует матрицу
        a = np.array([np.arange(length),cols]).T

        #смотрим форму массива, по оси 0 - это y, и по оси 1 - это x
        y, x = a.shape

        #берем половину массив, округляя в большую сторону
        col1 = a[:int(np.ceil(y/2))].copy()
        #оставщаяся часть
        col2 = a[int(np.ceil(y/2)):].copy()

        #определяем максимальную длину колонок, чтобы увеличить одну из колонок, если она получилась короче, например, когда 
        #кол-во столбцов было изначально 3, первая колонка будет длиной 2, а вторая 1
        length = max(col1.shape[0], col2.shape[0])

        #меняем размер колонок по максимальной длине
        col1.resize([length,2])
        col2.resize([length,2])

        #Приходится изворачиваться, у Питона нет ++/-- для инкремента/декремента. Те мелочи, за которые я не люблю Питон 
        i = 0
        while i < length:
            c1 = str(col1[i][0]).ljust(3) #первая колонка
            c1_len = len(c1) 
            c2 = repr(col1[i][1]).ljust(max_len+3) 
            c2_len = len(c2) 
            c3 = str(col2[i][0]).ljust(3) #вторая колонка
            c3_len = len(c3)
            c4 = repr(col2[i][1])
            print(c1 + c2 + c3 + c4)
            i += 1
    
    #тут пара функций для удобного сохранения из загрузки 
    @staticmethod 
    def load(filepath):
        import pickle as pkl
        with open(filepath, "rb") as f:
            return pkl.load(f)
    
    @staticmethod 
    def save(data, filepath):
        import pickle as pkl
        with open(filepath, "wb") as f:
            return pkl.dump(data, f, 2) #2 is protocol version
    
    @staticmethod 
    def read_excels(filepath):
        import pandas as pd
        return pd.read_excel(filepath, sheet_name=None, header=None, na_filter=False)

    @staticmethod 
    def read_excel(filepath):
        import pandas as pd
        return pd.read_excel(filepath, header=None, na_filter=False)

    @staticmethod 
    def read_zip(filepath):
        import zipfile as zip
        z = zip.ZipFile(filepath, mode='r')
        names = tuple(z.namelist())
        lst = []
        for f in names:
            lst += [z.open(f)]
        return (names, tuple(lst))
    
    @staticmethod 
    def df_filter_and(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
                
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_and(res, lst[i])
            return res
    
    @staticmethod 
    def df_filter_or(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
        
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_or(res, lst[i])
            return res
        
    
pp = Prepack       

In [423]:
df = pp.read_excel('2002.xls')
df2 = pp.read_excel('2003.xls')

In [424]:
df.iloc[8:12]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
8,,,,,,,,,,(тыс. руб.),,,(тыс. руб.),,,,,,,,,,,,,,,,
9,Наименование показателя,Код по бюджетной \nклассификации,Коды ФКР,Коды ЦСР,Коды ВР,Утверждено Федеральным законом «О федеральном ...,Уточненная сводная бюджетная роспись,Финансирование,Лимиты бюджетных обязательств,Исполнено,Процент исполнения уточненной сводной бюджетно...,Процент исполнения уточненной сводной бюджетно...,,,,,,,,,,,,,,,,,
10,,ППП,РзПр,ЦСР,ВР,,,,,,,,,,,,,,,,,,,,,,,,
11,1,2,3,4,5,6,7,8,8,9,10,10,,,,,,,,,,,,,,,,,


## Делаем заголовки столбцов

In [425]:
cols = []

# делаем назzвания столбцов в список
for i in [8,9]:
    cols = pp.list_concat(cols, list(df.iloc[i]))

for i, el in enumerate(cols):
    cols[i] = cols[i].replace('\n','')

pp.p2c(cols)

0  'Наименование показателя'                                                     15 ''
1  'Код по бюджетной классификации'                                              16 ''
2  'Коды ФКР'                                                                    17 ''
3  'Коды ЦСР'                                                                    18 ''
4  'Коды ВР'                                                                     19 ''
5  'Утверждено Федеральным законом «О федеральном бюджете на 2002 год»'          20 ''
6  'Уточненная сводная бюджетная роспись'                                        21 ''
7  'Финансирование'                                                              22 ''
8  'Лимиты бюджетных обязательств'                                               23 ''
9  '(тыс. руб.) Исполнено'                                                       24 ''
10 'Процент исполнения уточненной сводной бюджетной росписи по финансировагию'   25 ''
11 'Процент исполнения уточненной сводной б

In [426]:
df.columns = cols

In [427]:
cols = []

# делаем назzвания столбцов в список
for i in [8,9]:
    cols = pp.list_concat(cols, list(df2.iloc[i]))

for i, el in enumerate(cols):
    cols[i] = cols[i].replace('\n','')

pp.p2c(cols)

0  'Наименование показателя'                                                     15 ''
1  'Код по бюджетной классификации'                                              16 ''
2  'Коды ФКР'                                                                    17 ''
3  'Коды ЦСР'                                                                    18 ''
4  'Коды ВР'                                                                     19 ''
5  'Утверждено Федеральным законом «О федеральном бюджете на 2003 год»'          20 ''
6  'Уточненная сводная бюджетная роспись'                                        21 ''
7  'Финансирование'                                                              22 ''
8  'Лимиты бюджетных обязательств'                                               23 ''
9  '(тыс. руб.) Исполнено'                                                       24 ''
10 'Процент исполнения уточненной сводной бюджетной росписи по финансировагию'   25 ''
11 'Процент исполнения уточненной сводной б

In [428]:
df2.columns = cols

## отрезаем лишние столбцы

In [429]:
# данные только с 0 по 12 столбец
df = df.iloc[:,0:12]
df2 = df2.iloc[:,0:12]
df.tail(20)

,Наименование показателя,Код по бюджетной классификации,Коды ФКР,Коды ЦСР,Коды ВР,Утверждено Федеральным законом «О федеральном бюджете на 2002 год»,Уточненная сводная бюджетная роспись,Финансирование,Лимиты бюджетных обязательств,(тыс. руб.) Исполнено,Процент исполнения уточненной сводной бюджетной росписи по финансировагию,"Процент исполнения уточненной сводной бюджетной росписи,%"
4249,"Расходы на международные культурные, научные и...",722,0304,125,,,14175,14102.3,,14122.7,99.4871,99.631
4250,Участие в международных конференциях,722,0304,125,050,,14175,14102.3,,14122.7,99.4871,99.631
4251,Комитет Российской Федерации по финансовому мо...,723,,,,54930.9,102041,102035,,94717.7,99.9943,92.8235
4252,Государственное управление и местное самоуправ...,723,0100,,,54930.9,99520.6,99520.6,,92194.3,100,92.6384
4253,Деятельность финансовых и налоговых органов,723,0104,,,54930.9,99520.6,99520.6,,92194.3,100,92.6384
4254,Центральный аппарат,723,0104,037,,37144.3,77949.3,77949.3,,77830.3,100,99.8473
4255,Денежное содержание аппарата,723,0104,037,027,23062.1,32531.8,32531.8,,32412.8,100,99.6342
4256,Расходы на содержание аппарата,723,0104,037,029,14082.2,45417.5,45417.5,,45417.5,100,100
4257,Территориальные органы,723,0104,038,,17786.6,21571.3,21571.3,,14364,100,66.5885
4258,Денежное содержание аппарата,723,0104,038,027,10800.6,10504.3,10504.3,,6431.1,100,61.2235


## отрезам лишние строки

In [430]:
# для подготовки маски фильтра будет использована специальная функция, которая создает маски по количеству переданных условий,
# а потом складывает их через логическое И &. Т.е. остаются только те строки, которые удовлетворяют всем условиям вместе.
# В данном случае у нас 2 условия, поле наименование не число, а поле Код число.
f = pp.df_filter_and(df, {'Наименование показателя': 'isnotnum', 
                          'Код по бюджетной классификации': 'isnum'})
df_ = df[f].reset_index(drop=True)


In [431]:
f = pp.df_filter_and(df2, {'Наименование показателя': 'isnotnum', 
                          'Код по бюджетной классификации': 'isnum'})
df2_ = df2[f].reset_index(drop=True)


## Делим значения столбца Коды ФКР на 2 столбца: РЗ, ПР

In [432]:
new = df_['Коды ФКР'].str.extract(r'(\d\d)(\d\d)')
new.columns = ['РЗ','ПР']
df_ = df_.join(new)

In [433]:
new = df2_['Коды ФКР'].str.extract(r'(\d\d)(\d\d)')
new.columns = ['РЗ','ПР']
df2_ = df2_.join(new)

## Фильтруем, чтобы остались только данные по разделам

In [434]:
f = pp.df_filter_and(df_, {'Коды ВР': 'isnotnum', 
                            'Коды ЦСР': 'isnotnum',
                            'Уточненная сводная бюджетная роспись': 'isnum',
                            'ПР': '00'})
df_1 = df_[f]

In [435]:
f = pp.df_filter_and(df2_, {'Коды ВР': 'isnotnum', 
                            'Коды ЦСР': 'isnotnum',
                            'Уточненная сводная бюджетная роспись': 'isnum',
                            'ПР': '00'})
df2_1 = df2_[f]

In [436]:
df2_1.head(3)

,Наименование показателя,Код по бюджетной классификации,Коды ФКР,Коды ЦСР,Коды ВР,Утверждено Федеральным законом «О федеральном бюджете на 2003 год»,Уточненная сводная бюджетная роспись,Финансирование,Лимиты бюджетных обязательств,(тыс. руб.) Исполнено,Процент исполнения уточненной сводной бюджетной росписи по финансировагию,"Процент исполнения уточненной сводной бюджетной росписи,%",РЗ,ПР
1,Государственное управление и местное самоуправ...,020,0100,,,218210,218210,218201,,214688,99.9958,98.3859,01,00
11,Международная деятельность,020,0300,,,177366,202503,132590,,134334,65.4757,66.3368,03,00
22,Фундаментальные исследования и содействие науч...,020,0600,,,40738.3,40738.3,40738.3,,40738.3,100,100,06,00


In [437]:
pp.p2c(df2_1.columns)

0  'Наименование показателя'                                                     7  'Финансирование'
1  'Код по бюджетной классификации'                                              8  'Лимиты бюджетных обязательств'
2  'Коды ФКР'                                                                    9  '(тыс. руб.) Исполнено'
3  'Коды ЦСР'                                                                    10 'Процент исполнения уточненной сводной бюджетной росписи по финансировагию'
4  'Коды ВР'                                                                     11 'Процент исполнения уточненной сводной бюджетной росписи,%'
5  'Утверждено Федеральным законом «О федеральном бюджете на 2003 год»'          12 'РЗ'
6  'Уточненная сводная бюджетная роспись'                                        13 'ПР'


In [438]:
y2002 = df_1.iloc[:,[0,6,12]].groupby(['РЗ']).agg({'Наименование показателя': 'first',
                                           'Уточненная сводная бюджетная роспись': 'sum'})

In [439]:
y2003 = df2_1.iloc[:,[0,6,12]].groupby(['РЗ']).agg({'Наименование показателя': 'first',
                                           'Уточненная сводная бюджетная роспись': 'sum'})

In [440]:
yy = y2002.merge(y2003, on='РЗ', how='outer').reset_index()

In [441]:
yy.head(2)

,РЗ,Наименование показателя_x,Уточненная сводная бюджетная роспись_x,Наименование показателя_y,Уточненная сводная бюджетная роспись_y
0,01,Государственное управление и местное самоуправ...,58140558.3,Государственное управление и местное самоуправ...,67622533.0
1,02,Судебная власть,19900896.6,Судебная власть,25472082.5


In [442]:
yy.columns = ['РЗ','Наименование показателя 2002','тыс.р. 2002',
              'Наименование показателя 2003','тыс.р. 2003']

In [443]:
yy

,РЗ,Наименование показателя 2002,тыс.р. 2002,Наименование показателя 2003,тыс.р. 2003
0,01,Государственное управление и местное самоуправ...,58140558.3,Государственное управление и местное самоуправ...,67622533.0
1,02,Судебная власть,19900896.6,Судебная власть,25472082.5
2,03,Международная деятельность,41574577.2,Международная деятельность,38793959.3
3,04,Национальная оборона,2819655.7,Национальная оборона,223337084.9
4,05,Правоохранительная деятельность и обеспечение ...,58375369.3,Правоохранительная деятельность и обеспечение ...,192197986.3
5,06,Фундаментальные исследования и содействие науч...,29610544.9,Фундаментальные исследования и содействие науч...,40239736.7
6,07,"Промышленность, энергетика и строительство",103396876.6,"Промышленность, энергетика и строительство",71755990.3
7,08,Сельское хозяйство и рыболовство,28876289.4,Сельское хозяйство и рыболовство,32377348.5
8,09,Охрана окружающей природной среды и природных ...,9956333.3,"Охрана окружающей среды и природных ресурсов, ...",11373707.0
9,10,"Транспорт, связь и информатика",7495681.6,"Транспорт, связь и информатика",6571130.0
